# **TOPIC INTRODUCTION AND SOCIAL SCIENCE CONTEXT**

# Object and Color Detection for Analyzing Self-Representation of Republican and Democrat Members of Congress on Twitter

Name: Tunyanan Pimonbutpong 

Social media have been widely used by politicians for political communication (Stier, Bleier, Lietz, & Strohmaier, 2018). They enable them to circumvent traditional media and engage directly with voters, altering the relationship between political players and the news media. (Enli, 2017). This notebook is based on the concept of political identity ownership by Kreiss, D., Lawrence, R. G., & McGregor, S. C. (2020). Political identity ownership: Symbolic contests to represent members of the public. Social Media+ Society, 6(2), 2056305120926495.

Here, I use political identity ownership to propose that that political leaders and candidates use social media to construct and convey their social identities, and therefore their political identities. Candidates perform their own identities to align with groups whose support they seek.These acts strengthen separation from out-groups while also establishing each party as the "true" representative of specific social groups. (Sides et al., 2018).

My assumption for this is their political identity are perform through images, that the objects represent in their images might represent their personal qualities or their policy positions. Furthermore, I will also look at the dominant colors in the images of each party.

# Import Image Data 

In the first part, I will import the image dataset of republican and democrat members of congress. Here, I manually collected images through members of congress' Twitter accounts. For each member, I collected no more than two pictures. Therefore, all the images are posted by the members of congress themselves and not by the media. There are 94 images in total.




Here is the url of the zip files. Aftering downloading, I uploaded them directly to Google Colab and extracted the zip files using the following codes.

###Republican members
https://docs.google.com/uc?export=download&id=1vShdzocnXaklbl91Naq3CD_KJp26s9f5

###Democrat members 
https://docs.google.com/uc?export=download&id=1qmzBqt2pKmqFhOkYmnJgZPMe5q7WRCRX

In [9]:
import zipfile
#extract zip files 

with zipfile.ZipFile('democratsimages.zip' , 'r') as zip_ref:
    zip_ref.extractall()

with zipfile.ZipFile('republicansimages.zip' , 'r') as zip_ref:
    zip_ref.extractall()

After I extract the zip files into folders, I will conduct the object detection and color detection of these images 

# Object Detection

The next step, I will perform the object detection using facebook/detr-resnet-50 model on Hugging Face. I chose this model based on the performance of detection of objects compared with others model that I have tested on Hugging Face.

In [ ]:
!pip install -q transformers
!pip install -q timm
from transformers import pipeline
import pandas as pd 
model = "facebook/detr-resnet-50"
detectionmodel = pipeline("object-detection", model)

Firstly, the object detection will be performed on Republican members of congress images.

In [ ]:
import glob

rows_list = []
for img in sorted(glob.glob('republicansimages/image-*.jpg')):
  dict1 = {}
  res = detectionmodel(img)
  dict1.update({"image": img, "result": res})
  rows_list.append(dict1)

dfrepublican = pd.DataFrame(rows_list)

Next, I will conduct object detection on Democrat members of congress.

In [ ]:
import glob

rows_list = []
for img in sorted(glob.glob('democratsimages/pic-*.jpeg')):
  dict1 = {}
  res = detectionmodel(img)
  dict1.update({"image": img, "result": res})
  rows_list.append(dict1)

dfdemocrat = pd.DataFrame(rows_list)

# Object Detection Visualization

Next, I will unpack the results into a longer dataframe where each row is present the detected object.

In [ ]:
#Republicans

rows_list = []
for index, row in dfrepublican.iterrows():
  for i in row.result:
    dict1 = {}
    dict1.update({"image":row.image})
    dict1.update(i)
    rows_list.append(dict1)

df_republicans = pd.DataFrame(rows_list)

#Democrats

rows_list = []
for index, row in dfdemocrat.iterrows():
  for i in row.result:
    dict1 = {}
    dict1.update({"image":row.image})
    dict1.update(i)
    rows_list.append(dict1)

df_democrats = pd.DataFrame(rows_list)

Then, I will visuailize the results of object detection using the value_counts() function in pandas to count the occurrences of values in a given column of a DataFrame and plot value counts.





In [ ]:
df_republicans.label.value_counts().sort_values().plot(kind='barh') 

In [ ]:
df_democrats.label.value_counts().sort_values().plot(kind='barh') 

In the political context, objects may take on other abstract meanings, which, in the case of political figures, relate to their personal qualities or policy positions (Anastasopoulos et al., 2017). The bar plots of the objects appearing in the images of Republicans and Democrats show that some objects indicate each party's political owner identity.To begin, the images of Republican members, the conservative party, contain a greater number of ties than the images of Democratic members.The tie object here can represent the suit uniform, which is a male-specific object.

Furthermore, we will see that truck and airplane objects are detected in the images of the Republican members. In fact, these objects are related to the military. The object detection model cannot classify military equipment. As a result, the model simply categorizes these objects as truck and airplane.The truck objects here are military vehicles, and the airplanes are military helicopters. The Republican Party believes that the key to maintaining a safe and successful nation is maintaining a strong traditional military; thus, the party values a powerful national defense and a strong military in order to provide increased security and peace (Pew Research Center, 2018).

Looking at the objects in the Democratic members' images The objects are different from the Republicans, there are less tie objects appear in their members of congress pictures. The objects appear to have animals (i.e.,  cow and dog) and objects related to food (banana, apple, and orange). According to the party’s platform drafted by the Democratic National Committee (2020), the Democrats believe a powerful government can ensure welfare and equality for all. One of their policy priorities is to support farm workers and promote food security with the aim of developing and growing food systems (Democratic National Committee, 2020). They also aim to protect the humane treatment of farm animals and animal welfare (Democratic National Committee, 2020). Therefore, these objects in the images can emphasize the political identity of the party to seek support.

All in all, the objects present in the political figure images from Democrats and Republicans on social media are used with the purpose of communicating with the public, either to create the intention of persuading viewers to side with or against them. The objects represent symbolic meanings of Democrats' and Republicans' political identities.

# Color Detection

I will then further conduct the color detection to see the color representation in the images of both political parties.

In [1]:
#import required libraries
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os

#Read and Process Image Data for Color Identification

In [2]:
from imutils import paths
import cv2

In [13]:
image_democrats = list(paths.list_images("/content/democratsimages")) #path in Google Colab where the folder located 

image_republicans = list(paths.list_images("/content/republicansimages")) #path in Google Colab where the folder located 

#Image Manipulation



By default, OpenCV reads image in the sequence Blue Green Red (BGR). So, I will convert the rendering to Red Green Blue (RGB).

In [14]:
#Apply RGB to Democrat
for imagePath in image_democrats:
  image1 = cv2.imread(imagePath)
  image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

#Apply RGB to Republican
for imagePath in image_republicans:
  image2 = cv2.imread(imagePath)
  image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

I will resize the images to 600 x 400.  I resize the images in order to lessen the pixels which it will reduce the time needed to extract the colors from the image. Next, the KMeans alogorithm expects the input to be of two dimensions, so I will use Numpy’s reshape function to reshape the image data.

In [15]:
#Democrat
modified_democrats = cv2.resize(image1, (600, 400), interpolation = cv2.INTER_AREA)
modified_democrats = modified_democrats.reshape(modified_democrats.shape[0]*modified_democrats.shape[1], 3)

In [16]:
#Republican
modified_republicans = cv2.resize(image2, (600, 400), interpolation = cv2.INTER_AREA)
modified_republicans = modified_republicans.reshape(modified_republicans.shape[0]*modified_republicans.shape[1], 3)

#Color Identification 

I will then define a function that will convert RGB to hex for the purpose of use them as labels for a visualization (pie chart).

In [23]:
def rgb2hex(rgb):
    hex = "#{:02x}{:02x}{:02x}".format(int(rgb[0]), int(rgb[1]), int(rgb[2]))
    return hex

KMeans algorithm creates clusters based on the supplied count of clusters.  It will form clusters of colors and these clusters will be top colors. Here I chose 8 colors.

In [33]:
#Democrat
clt = KMeans(n_clusters = 6)
labels = clt.fit_predict(modified_democrats)

In [34]:
#Republican
clf = KMeans(n_clusters = 6)
labels2 = clf.fit_predict(modified_republicans)

# Color Detection Visualization 

Now, I will conduct visualization for the most 8 top colors of both political parties by using pie chart.

In [ ]:
#Democrat

counts = Counter(labels)

center_colors = clt.cluster_centers_
#get ordered colors by iterating through the keys
ordered_colors = [center_colors[i] for i in counts.keys()]
hex_colors = [rgb2hex(ordered_colors[i]) for i in counts.keys()]
rgb_colors = [ordered_colors[i] for i in counts.keys()]

#plot pie chart
plt.figure(figsize = (8, 6))
plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

#Republican

counts = Counter(labels2)

center_colors = clf.cluster_centers_
#get ordered colors by iterating through the keys
ordered_colors = [center_colors[i] for i in counts.keys()]
hex_colors = [rgb2hex(ordered_colors[i]) for i in counts.keys()]
rgb_colors = [ordered_colors[i] for i in counts.keys()]

#plot pie chart
plt.figure(figsize = (8, 6))
plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

From the top six color visualizations of Democrat and Republican party images, we will see that the color representation of Republican members of congress is clearly distinguishable from that of Democrat members of congress. Most of the top colors are related to the US military. The colors #baa786, #8c7d64, and #181718 represent the US military combat and service uniforms. The #484642 indicates the color of military equipment and vehicles. Therefore, the color representation of Republican members resembles the US Army uniform color scheme, while the colors in Democrat images do not show a clear pattern.

# References

Anastasopoulos, L. J., Badani, D., Lee, C., Ginosar, S., & Williams, J. R. (2017). Political image analysis with deep neural networks. University of Georgia, United States.

Democratic National Committee.(2020). BUILDING A STRONGER, FAIRER ECONOMY. https://democrats.org/where-we-stand/party-platform/building-a-stronger-fairer-economy/

Kreiss, D., Lawrence, R. G., & McGregor, S. C. (2020). Political identity ownership: Symbolic contests to represent members of the public. Social Media+ Society, 6(2), 2056305120926495.

Pew Research Center.(2017).Conflicting Partisan Priorities for U.S. Foreign Policy. https://www.pewresearch.org/politics/2018/11/29/conflicting-partisan-priorities-for-u-s-foreign-policy/

Stier, S., Bleier, A., Lietz, H., & Strohmaier, M. (2018). Election campaigning on social media: Politicians, audiences, and the mediation of political communication on Facebook and Twitter. Political communication, 35(1), 50-74.